In [2]:
import subprocess
import os
from pathlib import Path
import xarray as xr

import geopandas as gpd
import numpy as np

import cdsapi
import matplotlib.pyplot as plt
import cartopy.crs as crs
import xarray as xr
import seaborn as sns

import os
import subprocess
from functools import partial
from collections import namedtuple

%matplotlib inline
plt.rcParams['figure.figsize'] = 12, 6

In [7]:
# Set working directory as project root
if os.getcwd().split('/')[-1] != "esowc_notes":
    os.chdir('..')
    
assert os.getcwd().split('/')[-1] == "esowc_notes", f"Working directory should be the root (), currently: {os.getcwd()}"
print("Current Directory: ",os.getcwd())

Current Directory:  /Users/tommylees/github/esowc_notes


In [8]:
from src.api_helpers import Region

# Download ESA CCI Landcover as netcdf
[CEDA Data Store](http://data.ceda.ac.uk/neodc/esacci/land_cover/data/land_cover_maps/v1.6.1)


OR: from the [original ESA CCI website](http://maps.elie.ucl.ac.be/CCI/viewer/download.php)
NOTE: 2.3GB of data! Need to find a way of recompressing this (global - 300m product)

In [9]:
if os.getcwd().split('/')[2] == 'tommylees':
    data_dir = Path("/Volumes/Lees_Extend/data/ecmwf_sowc/")
else:
    data_dir = Path('./data')
    
print(data_dir)

/Volumes/Lees_Extend/data/ecmwf_sowc


In [10]:
[f for f in data_dir.glob('*')]

[PosixPath('/Volumes/Lees_Extend/data/ecmwf_sowc/chirps_kenya.nc'),
 PosixPath('/Volumes/Lees_Extend/data/ecmwf_sowc/chirps_kenya2.nc'),
 PosixPath('/Volumes/Lees_Extend/data/ecmwf_sowc/countries_shp'),
 PosixPath('/Volumes/Lees_Extend/data/ecmwf_sowc/countries_shp.zip'),
 PosixPath('/Volumes/Lees_Extend/data/ecmwf_sowc/era5_test.grib'),
 PosixPath('/Volumes/Lees_Extend/data/ecmwf_sowc/ESACCI-LC-L4-LCCS-Map-300m-P1Y-2015-v2.0.7b.nc'),
 PosixPath('/Volumes/Lees_Extend/data/ecmwf_sowc/ESACCI-LC-L4-LCCS-Map-300m-P1Y-2015-v2.0.7b.nc.zip'),
 PosixPath('/Volumes/Lees_Extend/data/ecmwf_sowc/ESACCI-LC-Legend.csv'),
 PosixPath('/Volumes/Lees_Extend/data/ecmwf_sowc/GRUN_v1_GSWP3_WGS84_05_1902_2014.nc'),
 PosixPath('/Volumes/Lees_Extend/data/ecmwf_sowc/kenya_precip_JanJune.grib'),
 PosixPath('/Volumes/Lees_Extend/data/ecmwf_sowc/kenya_rainfall.nc'),
 PosixPath('/Volumes/Lees_Extend/data/ecmwf_sowc/kenya_rainfall1.nc'),
 PosixPath('/Volumes/Lees_Extend/data/ecmwf_sowc/kenya_rainfall2.nc'),
 PosixP

In [11]:
if not (data_dir/"ESACCI-LC-L4-LCCS-Map-300m-P1Y-2015-v2.0.7b.nc.zip").exists():
    process = subprocess.Popen("chmod +x example_scripts/download_cci_landcover_2005.sh".split(), stdout=subprocess.PIPE)
    process = subprocess.Popen('example_scripts/download_cci_landcover_2005.sh ./data', shell=True)
    output, error = process.communicate()
else:
    print('File already exists!')

File already exists!


In [12]:
lc = xr.open_dataset(data_dir/"ESACCI-LC-L4-LCCS-Map-300m-P1Y-2015-v2.0.7b.nc", chunks={'lat':1000,'lon':1000})
lc_legend = pd.read_csv(data_dir/'ESACCI-LC-Legend.csv',sep=';')

# get geog location & turn into a subset string
kenya_region = Region(
    name='kenya',
    lonmin=33.501,
    lonmax=42.283,
    latmin=-5.202,
    latmax=6.002,
)

lc = lc.sel(lon=slice(kenya_region.lonmin,kenya_region.lonmax), lat=slice(kenya_region.latmin,kenya_region.latmax))
lc

OSError: [Errno -101] NetCDF: HDF error: b'/Volumes/Lees_Extend/data/ecmwf_sowc/ESACCI-LC-L4-LCCS-Map-300m-P1Y-2015-v2.0.7b.nc'

In [4]:
# create lookup dictionary from pd.DataFrame
lookup = lc_legend[['NB_LAB','LCCOwnLabel']]
lookup = dict(zip(lookup.iloc[:,0], lookup.iloc[:,1]))

from engineering.eng_utils import get_lookup_val, drop_nans_and_flatten, create_flattened_dataframe_of_values

lc_2 = get_lookup_val(xr_obj=lc, variable='esa_cci_landcover',
        new_variable='lc_string', lookup_dict=lookup
)

NameError: name 'lc_legend' is not defined

In [ ]:
from src.preprocessing_utils import convert_to_same_grid, select_same_time_slice

# convert_to_same_grid(reference_ds, ds, method="nearest_s2d")

# [PLOTTING STUFF](https://annefou.github.io/metos_python/04-plotting/)